<a href="https://colab.research.google.com/github/piainmaster/ConvNext_adapted-v2/blob/pn%2FResNeXt-ify_depthconv/Finetune_ConvNext_on_CIFAR10_using_W%26B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use this notebook to finetune a ConvNeXt-tiny model on CIFAR 10 dataset. The [official ConvNeXt repository](https://github.com/facebookresearch/ConvNeXt) is instrumented with [Weights and Biases](https://wandb.ai/site). You can now easily log your train/test metrics and version control your model checkpoints to Weigths and Biases

# ⚽️ Installation and Setup

The following installation instruction is based on [INSTALL.md](https://github.com/facebookresearch/ConvNeXt/blob/main/INSTALL.md) provided by the official ConvNeXt repository.

In [1]:
!pip install -qq torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -qq wandb timm==0.4.12 six tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.11.0+cu113 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.11.0+cu113 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 17.1 MB/s

Download the official ConvNeXt respository.

In [2]:
!git clone -b main https://github.com/piainmaster/ConvNext_adapted-v2/ ConvNeXt_finetune
!git clone -b pn/ResNeXt-ify_depthconv https://github.com/piainmaster/ConvNext_adapted-v2/ ConvNeXt_revertResNeXt-ify_depthconv


Cloning into 'ConvNeXt_finetune'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (359/359), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 359 (delta 199), reused 343 (delta 191), pack-reused 0
Receiving objects: 100% (359/359), 93.50 KiB | 5.50 MiB/s, done.
Resolving deltas: 100% (199/199), done.
Cloning into 'ConvNeXt_revertResNeXt-ify_depthconv'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (359/359), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 359 (delta 199), reused 343 (delta 191), pack-reused 0
Receiving objects: 100% (359/359), 93.50 KiB | 5.50 MiB/s, done.
Resolving deltas: 100% (199/199), done.


# 🏀 Download the Dataset

We will be finetuning on CIFAR-10 dataset. To use any custom dataset (CIFAR-10 here) the format of the dataset should be as shown below:

```
/path/to/dataset/
  train/
    class1/
      img1.jpeg
    class2/
      img2.jpeg
  val/
    class1/
      img3.jpeg
    class2/
      img4.jpeg
```

I have used this [repository](https://github.com/YoongiKim/CIFAR-10-images) that has the CIFAR-10 images in the required format.

In [3]:
!git clone https://github.com/YoongiKim/CIFAR-10-images

Cloning into 'CIFAR-10-images'...
remote: Enumerating objects: 60027, done.
remote: Total 60027 (delta 0), reused 0 (delta 0), pack-reused 60027
Receiving objects: 100% (60027/60027), 19.94 MiB | 37.06 MiB/s, done.
Resolving deltas: 100% (59990/59990), done.
Updating files: 100% (60001/60001), done.


# 🏈 Download Pretrained Weights

We will be finetuning the ConvNeXt Tiny model pretrained on ImageNet 1K dataset.

In [4]:
%cd ConvNeXt_revertResNeXt-ify_depthconv/
!wget https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth

/content/ConvNeXt_revertResNeXt-ify_depthconv
--2024-02-22 10:00:06--  https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.224.14.65, 13.224.14.69, 13.224.14.100, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.224.14.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114414741 (109M) [binary/octet-stream]
Saving to: ‘convnext_tiny_1k_224_ema.pth’

convnext_tiny_1k_22 100%[===================>] 109.11M   181MB/s    in 0.6s    

2024-02-22 10:00:07 (181 MB/s) - ‘convnext_tiny_1k_224_ema.pth’ saved [114414741/114414741]



In [5]:
!git pull

Already up to date.


# 🎾 Train with Weights and Biases

If you want to log the train and evaluation metrics using Weights and Biases pass `--enable_wandb true`.

You can also save the finetuned checkpoints as version controlled W&B [Artifacts](https://docs.wandb.ai/guides/artifacts) if you pass `--wandb_ckpt true`.



In [ ]:
!python main.py --epochs 100 \
                --batch_size 64 \
                --model convnext_tiny \
                --data_set image_folder \
                --data_path ../CIFAR-10-images/train \
                --eval_data_path ../CIFAR-10-images/test \
                --nb_classes 10 \
                --num_workers 8 \
                --warmup_epochs 0 \
                --save_ckpt true \
                --output_dir model_ckpt \
                --cutmix 0 \
                --mixup 0 --lr 4e-4 \
                --enable_wandb true --wandb_ckpt true

Not using distributed mode
Namespace(batch_size=64, epochs=100, update_freq=1, model='convnext_tiny', drop_path=0, input_size=224, layer_scale_init_value=1e-06, model_ema=False, model_ema_decay=0.9999, model_ema_force_cpu=False, model_ema_eval=False, opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=None, momentum=0.9, weight_decay=0.05, weight_decay_end=None, lr=0.0004, layer_decay=1.0, min_lr=1e-06, warmup_epochs=0, warmup_steps=-1, color_jitter=0.4, aa='rand-m9-mstd0.5-inc1', smoothing=0.1, train_interpolation='bicubic', crop_pct=None, reprob=0.25, remode='pixel', recount=1, resplit=False, mixup=0.0, cutmix=0.0, cutmix_minmax=None, mixup_prob=1.0, mixup_switch_prob=0.5, mixup_mode='batch', finetune='', head_init_scale=1.0, model_key='model|module', model_prefix='', data_path='../CIFAR-10-images/train', eval_data_path='../CIFAR-10-images/test', nb_classes=10, imagenet_default_mean_and_std=True, data_set='image_folder', output_dir='model_ckpt', log_dir=None, device='cuda', seed=0,

# 🏐 Conclusion

* **The above setting gives a top-1 accuracy of ~95%.**
* The ConvNeXt repository comes with modern training regimes and is easy to finetune on any dataset.
* The finetune model achieves competitive results.

* By passing two arguments you get the following:

  * Repository of all your experiments (train and test metrics) as a [W&B Project](https://docs.wandb.ai/ref/app/pages/project-page). You can easily compare experiments to find the best performing model.
  * Hyperparameters (Configs) used to train individual models.
  * System (CPU/GPU/Disk) metrics.
  * Model checkpoints saved as W&B Artifacts. They are versioned and easy to share.

  Check out the associated [W&B run page](https://wandb.ai/ayut/convnext/runs/16vi9e31). $→$